<h1 align="center">Tracking AIS Reports Using Stone Soup</h1>
<h3 align="center">Demonstrating the capabilities of Stone Soup at Beta release using recorded AIS data</h3>

In this notebook we will load a CSV file of AIS data to use as detections in a Stone Soup tracker. We will build the tracker from individual components and display the track output on a map.

Initially we will import the libraries we will need for tracking, conversions, and map plotting.

In [1]:
import datetime
from collections import defaultdict
from itertools import cycle

import folium
import numpy as np
import utm

Next we will load in our AIS data, saved as a CSV, as detections using the Stone Soup CSV reader.

In [2]:
from stonesoup.reader.generic import CSVDetectionReader
detector = CSVDetectionReader(
    "SolentAIS_20160112_130211.csv",
    state_vector_fields=("Longitude_degrees", "Latitude_degrees"),
    time_field="Time")

We use a feeder class to mimic a detector, passing our detections into the tracker in a time series.

In [3]:
# Limit updates to one detection per MMSI every minute
from stonesoup.feeder.time import TimeSyncFeeder
detector = TimeSyncFeeder(detector, time_window=datetime.timedelta(seconds=60))

from stonesoup.feeder.filter import MetadataReducer
detector = MetadataReducer(detector, 'MMSI')

In this instance, we want to convert the Latitude/Longitude information from the AIS file to UTM-WGS84 for plotting.

In [4]:
# Convert to local UTM Zone
from stonesoup.feeder.geo import LongLatToUTMConverter
detector = LongLatToUTMConverter(detector)

Now we begin to build our tracker from Stone Soup components. The first compenent we will build is a linear transition model. We create a two-dimensional transition model by combining two individual one-dimensional constant velocity models.

In [5]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel,\
                                               ConstantVelocity
transition_model = CombinedLinearGaussianTransitionModel(
    (ConstantVelocity(10), ConstantVelocity(10)))

Next we build a measurement model to describe the uncertainty on our detections.

In [6]:
from stonesoup.models.measurement.linear import LinearGaussian
measurement_model = LinearGaussian(
    ndim_state=4, mapping=[0, 2], noise_covar=np.diag([10, 10]))

With these models we can now create the predictor and updater components of the tracker, passing in the respective models.

In [7]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

In [8]:
from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

To associate our detections to track objects generate hypotheses using a hypothesiser. In this case we are using a simple Euclidean distance measure to score our hypotheses.

In [9]:
from stonesoup.hypothesiser.filtered import FilteredDetectionsHypothesiser
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
measure = Mahalanobis()
hypothesiser = FilteredDetectionsHypothesiser(
    DistanceHypothesiser(predictor, updater, measure, missed_distance=3),
    metadata_filter="MMSI"
)

We will use a nearest-neighbour association algorithm, passing in the Euclidean distance hypothesiser built in the previous step.

In [10]:
from stonesoup.dataassociator.neighbour import NearestNeighbour
data_associator = NearestNeighbour(hypothesiser)

We must have a method to initiate tracks. For this we will create an initiator component and have it generate a Gaussian distribution.

In [11]:
from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import LinearMeasurementInitiator
initiator = LinearMeasurementInitiator(
    GaussianState([[0], [0], [0], [0]], np.diag([0, 15, 0, 15])),
    measurement_model)

As well as an initiator we must also have a deleter. This deleter removes tracks which haven't been updated for a defined time period.

In [12]:
from stonesoup.deleter.time import UpdateTimeDeleter
deleter = UpdateTimeDeleter(datetime.timedelta(minutes=10))

With all the individual components specified we can now build our tracker. This is as simple as passing in the components.

In [13]:
from stonesoup.tracker.simple import MultiTargetTracker
tracker = MultiTargetTracker(
    initiator=initiator,
    deleter=deleter,
    detector=detector,
    data_associator=data_associator,
    updater=updater,
)

Our tracker is built and our detections have been read in from the CSV file, now we set the tracker to work. This is done by initiating a loop to generate tracks at each time interval.

In [14]:
tracks = set()
for step, (time, ctracks) in enumerate(tracker.tracks_gen(), 1):
    tracks.update(ctracks)
    if not step % 10:
        print("Step: {} Time: {}".format(step, time))

Step: 10 Time: 2016-01-12 13:12:11.218000
Step: 20 Time: 2016-01-12 13:22:11.218000
Step: 30 Time: 2016-01-12 13:32:11.218000
Step: 40 Time: 2016-01-12 13:42:11.218000
Step: 50 Time: 2016-01-12 13:52:11.218000
Step: 60 Time: 2016-01-12 14:02:11.218000
Step: 70 Time: 2016-01-12 14:12:11.218000
Step: 80 Time: 2016-01-12 14:22:11.218000


The tracker has now run over the full data set and produced an output track list. In this data set, from the below, we can see that we generated different numer of tracks from the number of unique vessels/MMSIs.

In [15]:
len(tracks)

115

In [16]:
len({track.metadata['MMSI'] for track in tracks})

91

We will use the Folium python library to display these tracks on a map. The icons can be clicked on to reveal the track metadata. (Tracks with same MMSI will be same colour, but colour may be used for multiple MMSIs)

In [17]:
colour_iter = iter(cycle(
    ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
     'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
     'darkpurple', 'pink', 'lightblue', 'lightgreen',
     'gray', 'black', 'lightgray']))
colour = defaultdict(lambda: next(colour_iter))

m = folium.Map(location=[50.75, -1], zoom_start=10)
for track in tracks:
    points = np.array([
        utm.to_latlon(
            *state.state_vector[measurement_model.mapping, 0],
            detector.zone_number, northern=detector.northern, strict=False)
        for state in track])
    folium.PolyLine(points, color=colour[track.metadata.get('MMSI')]).add_to(m)
    folium.Marker(utm.to_latlon(*track.state_vector[measurement_model.mapping, 0],
                                detector.zone_number, northern=detector.northern, strict=False),
                  icon=folium.Icon(icon='fa-ship', prefix="fa", color=colour[track.metadata.get('MMSI')]),
                  popup="\n".join("{}: {}".format(key, value) for key, value in track.metadata.items())).add_to(m)

In [18]:
m